In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json


#Loading data
columns_name = ['id', 'date', 'latitude', 'longitude' ]
df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")

df['week'] = df['date'].dt.isocalendar().week

In [2]:
df.isnull().sum()

id           0
date         0
latitude     0
longitude    0
week         0
dtype: int64

In [3]:
df.duplicated().sum()

17896957

In [4]:
id_week_ori = df.groupby('id')['week'].unique()
list_id_ori = df["id"].unique()
print(np.sort(list_id_ori), len(list_id_ori))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69


In [5]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

In [6]:
print(mean_lat_ori)
print(mean_long_ori)
print(mean_date_ori) 
print(count_ligne_ori)
print(mean_hour_ori)

id   week
1    10      4.827873
     11      4.876658
     12      3.315934
     13      4.411797
     14      4.875356
               ...   
107  10      4.870500
     11      4.764978
     12      4.879813
110  10     -0.119641
     11      2.344631
Name: latitude, Length: 449, dtype: float64
id   week
1    10      45.592419
     11      45.781152
     12      44.737925
     13      45.429105
     14      45.774198
               ...    
107  10      45.774327
     11      45.862149
     12      45.786341
110  10      48.404082
     11      48.884047
Name: longitude, Length: 449, dtype: float64
id   week
1    10     2015-03-05 21:39:07.527484928
     11     2015-03-13 14:25:11.006843136
     12     2015-03-19 23:50:54.719796480
     13     2015-03-27 05:09:54.252334592
     14     2015-04-02 16:30:12.192103680
                         ...             
107  10     2015-03-07 04:27:02.706065408
     11     2015-03-13 22:06:22.832617984
     12     2015-03-17 11:12:29.716608000
110  10 

In [7]:
for week, group in df.groupby('week'):
    print(f'Week {week}:')
    df_week = df[df['week'] == week]
    count_id_ori_week = df_week['id'].value_counts().reset_index()
    count_id_ori_week.columns = ['id', 'count']
    count_id_ori_week = count_id_ori_week.sort_values(by='count', ascending=True)
    list_count_id_ori_week = count_id_ori_week['id'].tolist()
    print(list_count_id_ori_week, len(list_count_id_ori_week))
    print()

Week 10:
[98, 18, 63, 107, 5, 66, 78, 32, 30, 77, 68, 29, 48, 110, 35, 2, 13, 83, 73, 60, 67, 25, 71, 16, 34, 24, 49, 1, 39, 41, 21, 89, 23, 72, 43, 54, 59, 26, 9, 28, 17, 37, 75, 58, 44, 15, 51, 31, 8, 14, 7, 52, 50, 27, 55, 87, 36, 11, 69, 42, 4, 38, 62, 6] 64

Week 11:
[29, 98, 5, 48, 58, 30, 77, 107, 110, 21, 78, 63, 68, 54, 25, 49, 37, 6, 1, 75, 60, 16, 13, 23, 34, 67, 89, 2, 83, 14, 41, 15, 24, 11, 52, 36, 9, 26, 44, 59, 73, 43, 8, 31, 28, 7, 17, 55, 72, 62, 87, 4, 38, 51, 50, 69, 27, 42, 53] 59

Week 12:
[34, 63, 38, 84, 65, 16, 55, 71, 78, 68, 70, 15, 77, 30, 107, 1, 9, 24, 44, 18, 32, 53, 6, 81, 66, 41, 52, 8, 2, 60, 75, 49, 26, 72, 67, 69, 43, 42, 51, 62, 31, 59, 13, 73, 28, 50, 87, 7, 17, 27, 4] 51

Week 13:
[44, 71, 77, 29, 70, 9, 84, 43, 54, 18, 41, 63, 49, 35, 52, 60, 24, 68, 83, 30, 32, 78, 66, 1, 51, 65, 17, 75, 67, 2, 8, 13, 28, 59, 72, 31, 73, 81, 62, 69, 7, 26, 27, 42, 53, 6, 50, 4] 48

Week 14:
[71, 17, 52, 78, 9, 84, 29, 35, 54, 70, 68, 53, 27, 32, 63, 66, 49, 75, 

In [8]:
for week, group in df.groupby('week'):
    print(f'Week {week}:')
    df_week = df[df['week'] == week]    
    count_id_ori_each_week = df_week['id'].value_counts().reset_index()
    count_id_ori_each_week.columns = ['id', 'count']

    print(count_id_ori_each_week)

Week 10:
     id   count
0     6  238005
1    62  209941
2    38  200805
3     4  198951
4    42  183845
..  ...     ...
59    5    4110
60  107    3215
61   63    1221
62   18     923
63   98     444

[64 rows x 2 columns]
Week 11:
     id   count
0    53  311171
1    42  288453
2    27  256638
3    69  243740
4    50  241614
5    51  216169
6    38  208422
7     4  195647
8    87  195320
9    62  190390
10   72  163536
11   55  159087
12   17  149269
13    7  140274
14   28  137035
15   31  136774
16    8  136629
17   43  134535
18   73  122894
19   59  121514
20   44  116557
21   26  113238
22    9  109638
23   36   91063
24   52   84146
25   11   76972
26   24   74595
27   15   67135
28   41   62907
29   14   57700
30   83   57338
31    2   56993
32   89   54949
33   67   53496
34   34   51920
35   23   40703
36   13   39178
37   16   38757
38   60   35893
39   75   35739
40    1   32587
41    6   29598
42   37   26438
43   49   23574
44   25   20949
45   54   19377
46   68   16790

In [2]:
#Random minute : returns a new date by adding or subtracting the random number of minutes, depending on whether the hour remains the same.
def random_minute(date):
    rand_min = random.randint(-29,29)
    return date - dt.timedelta(minutes=rand_min) if date.hour == (date - dt.timedelta(minutes=rand_min)).hour else date + dt.timedelta(minutes=rand_min)

#Change date to random (same week)
def change_date(date):
    rand_day = int(np.random.choice([-1,0,1], p=[0.02, 0.96, 0.02]))
    return date - dt.timedelta(days=rand_day) if date.isocalendar()[1] == (date - dt.timedelta(days=rand_day)).isocalendar()[1] else date + dt.timedelta(days=rand_day)

In [3]:
#Set id to 'DEL' with lines duplicated and retain only the first occurrence of each unique combination of values in these columns.
id_time = df.drop_duplicates(['id', 'date', 'latitude', 'longitude'])
print(id_time.shape)

index_not_change = np.array(id_time.index)
df.loc[~df.index.isin(index_not_change) , 'id'] = "DEL"

print(df.head())

(16654892, 5)
    id                date  latitude  longitude  week
0    1 2015-03-04 00:35:00  4.870147  45.772140    10
1    1 2015-03-04 00:35:00  4.870218  45.772095    10
2    1 2015-03-04 00:35:00  4.870210  45.772072    10
3  DEL 2015-03-04 00:35:00  4.870210  45.772072    10
4  DEL 2015-03-04 00:35:00  4.870210  45.772072    10


In [5]:
df

,id,date,latitude,longitude,week
0,1,2015-03-04 00:35:00,4.870147,45.772140,10
1,1,2015-03-04 00:35:00,4.870218,45.772095,10
2,1,2015-03-04 00:35:00,4.870210,45.772072,10
3,DEL,2015-03-04 00:35:00,4.870210,45.772072,10
4,DEL,2015-03-04 00:35:00,4.870210,45.772072,10
...,...,...,...,...,...
34551844,110,2015-03-12 16:23:00,2.343094,48.891650,11
34551845,110,2015-03-12 16:23:00,2.343094,48.891650,11
34551846,110,2015-03-12 16:23:00,2.343094,48.891649,11
34551847,110,2015-03-12 16:23:00,2.343094,48.891649,11


In [6]:
#Change lat-long
df['latitude'] = df['latitude'] + 0.005 * (1 / (1 + np.tan(df['latitude']) * np.tan(df['latitude'])))
df['longitude'] = df['longitude'] + 0.005 * (1 / (1 + np.tan(df['longitude']) * np.tan(df['longitude'])))

In [8]:
#Apply random_minute for the ids not DEL
df.loc[df['id'] != 'DEL', 'date'] = df.loc[df['id'] != 'DEL', 'date'].apply(random_minute)

#Apply change_date for the ids not DEL
df.loc[df['id'] != 'DEL', 'date'] = df.loc[df['id'] != 'DEL', 'date'].apply(change_date) 

In [15]:
for week, group in id_time.groupby('week'):
    print(f'Week {week}:')
    id_time_week = id_time[id_time['week'] == week]    
    count_id_ori_each_week = id_time_week['id'].value_counts().reset_index()
    count_id_ori_each_week.columns = ['id', 'count']

    print(count_id_ori_each_week)

Week 10:
     id   count
0     6  222194
1     4  117768
2    11   97240
3    38   87014
4    42   66530
..  ...     ...
59   66    2992
60  107    2210
61   63     921
62   18     752
63   98     344

[64 rows x 2 columns]
Week 11:
     id   count
0    50  126067
1    69  121589
2    27  116479
3    51  113625
4     4  109532
5    42   99890
6    38   97228
7    53   93147
8    72   92751
9     8   77954
10   28   74121
11   55   67150
12    9   66603
13   43   62098
14    7   57972
15   26   52211
16   59   51345
17   17   49828
18   31   48240
19    2   44983
20   44   44204
21   11   41443
22   24   41126
23   87   40434
24   15   38426
25   52   38146
26   14   36644
27   89   32242
28   67   32093
29   73   30715
30   62   26404
31    6   26272
32   83   25460
33   23   24810
34   34   24373
35    1   22629
36   36   22407
37   13   22142
38   41   22126
39   75   20946
40   25   15693
41   60   13751
42   37   12962
43   68   12558
44   54   10233
45   63   10210
46   49    9690

In [9]:
#Id anonymisation
id_time['new_id'] =  id_time['id'] + id_time['week']

id_time['new_id'] = id_time['new_id'].apply(lambda x: abs(int(1000 * np.tan(x))))
df.loc[df['id'] != 'DEL', 'id'] = id_time['new_id']

C:\Users\Phuoc Le\AppData\Local\Temp\ipykernel_10860\573553612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_time['new_id'] =  id_time['id'] + id_time['week']
C:\Users\Phuoc Le\AppData\Local\Temp\ipykernel_10860\573553612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_time['new_id'] = id_time['new_id'].apply(lambda x: abs(int(1000 * np.tan(x))))


In [10]:
df

,id,date,latitude,longitude,week
0,225950,2015-03-04 00:40:00,4.870270,45.772376,10
1,225950,2015-03-04 00:18:00,4.870342,45.772331,10
2,225950,2015-03-04 00:51:00,4.870334,45.772308,10
3,DEL,2015-03-04 00:35:00,4.870334,45.772308,10
4,DEL,2015-03-04 00:35:00,4.870334,45.772308,10
...,...,...,...,...,...
34551844,20524,2015-03-12 16:35:00,2.345528,48.891841,11
34551845,20524,2015-03-12 16:25:00,2.345528,48.891841,11
34551846,20524,2015-03-12 16:26:00,2.345528,48.891841,11
34551847,20524,2015-03-12 16:26:00,2.345528,48.891841,11


In [30]:
id_new_week_ori = new_df.groupby('id')['week'].unique()
list_new_id_ori = new_df["id"].unique()
print(np.sort(list_new_id_ori), len(list_new_id_ori))

[8 17 26 35 106 115 124 133 151 160 169 178 188 252 262 271 281 300 310
 320 329 339 400 410 420 431 441 463 473 484 495 506 587 599 611 623 635
 661 673 686 699 713 796 810 825 840 855 887 903 919 935 952 1059 1078
 1097 1117 1137 1178 1200 1221 1244 1415 1442 1470 1498 1527 1588 1619
 1652 1685 1995 2040 2086 2134 2237 2291 2347 2406 2468 2987 3077 3172
 3273 3493 3614 3743 3880 4027 5191 5451 5737 6053 6405 7244 7750 8330
 9003 9792 20524 25092 32268 45183 75313 225950] 104


In [31]:
for week, group in new_df.groupby('week'):
    print(f'Week {week}:')
    new_df_week = new_df[new_df['week'] == week]    
    count_id_ori_each_week = new_df_week['id'].value_counts().reset_index()
    count_id_ori_each_week.columns = ['id', 'count']

    print(count_id_ori_each_week)

Week 10:
      id   count
0    300  222194
1   7244  117768
2   1527   97240
3   1200   87014
4   6053   66530
..   ...     ...
59   686    2992
60   952    2210
61   919     921
62   281     752
63  2468     344

[64 rows x 2 columns]
Week 11:
       id   count
0    3743  126067
1    9003  121589
2     310  116479
3    1097  113625
4     855  109532
5     431   99890
6    3172   97228
7    2347   93147
8    3880   92751
9     151   77954
10   3614   74121
11     26   67150
12   2237   66603
13    673   62098
14   1137   57972
15    840   52211
16   1221   51345
17    281   49828
18   2291   48240
19    463   44983
20  45183   44204
21      8   41443
22    473   41126
23    699   40434
24   1178   38426
25    169   38146
26    133   36644
27    587   32242
28    599   32093
29   1078   30715
30    919   26404
31   3493   26272
32    252   25460
33    623   24810
34   1619   24373
35    635   22629
36    124   22407
37   2134   22142
38   6053   22126
39   2406   20946
40   7750   15693

In [11]:
df = df.drop('week', axis = 1)

In [12]:
df

,id,date,latitude,longitude
0,225950,2015-03-04 00:40:00,4.870270,45.772376
1,225950,2015-03-04 00:18:00,4.870342,45.772331
2,225950,2015-03-04 00:51:00,4.870334,45.772308
3,DEL,2015-03-04 00:35:00,4.870334,45.772308
4,DEL,2015-03-04 00:35:00,4.870334,45.772308
...,...,...,...,...
34551844,20524,2015-03-12 16:35:00,2.345528,48.891841
34551845,20524,2015-03-12 16:25:00,2.345528,48.891841
34551846,20524,2015-03-12 16:26:00,2.345528,48.891841
34551847,20524,2015-03-12 16:26:00,2.345528,48.891841


In [13]:
# Save file
df.to_csv('trigonometry_2.csv', sep='\t', index = False, header = False, columns = None)